# Installing and Importing the required libraries

In [1]:
#pip install langchain langchain-experimental openai pymysql

In [2]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage

# Using the OpenAI_API_KEY

In [3]:
OPENAI_API_KEY = ""
llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)

C:\Users\lavanya\AppData\Local\Temp\ipykernel_18792\450203358.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)


# Connecting to MYSQL WORKBENCH

In [4]:
host = 'localhost'
port = '3306'
username= 'root'
password='root'
database_schema = 'movies'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri,include_tables=['telugumovies_dataset'])
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)

# Creating two functions one for retrieving and one for generating

In [5]:
def retrieve_from_db(query:str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [6]:
db_context = retrieve_from_db("What is the total no of movies available in the table?")

C:\Users\lavanya\AppData\Local\Temp\ipykernel_18792\2548019726.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  db_context = db_chain(query)




> Entering new SQLDatabaseChain chain...
What is the total no of movies available in the table?
SQLQuery:SELECT COUNT(`Movie`) AS TotalMovies FROM telugumovies_dataset;
SQLResult: [(1397,)]
Answer:Total number of movies available in the table is 1397.
> Finished chain.


In [7]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    system_message = """You are a movie database expert specializing in Telugu Cinema. Your task is to answer users questions by providing relevant information from a database of Telugu movies.The database contains the following information for each movie:
    -Name of the movie
    -Year of release
    -Certificate given by the censor board
    -Movie genre
    -A brief description or plot of the movie
    -Duration in minutes
    -IMDb Rating
    -Number of people who rated the movie"""
    human_qry_template = HumanMessagePromptTemplate.from_template("""Input:{human_input} Context:{db_context} Output: """)
    messages = [SystemMessage(content= system_message),human_qry_template.format(human_input=query,db_context=db_context)]
    response = llm(messages).content
    return response

# Using various prompts to generate SQL Queries and generate answers to the questions provided

In [8]:
generate("How many unique movies start with the letter B?")



> Entering new SQLDatabaseChain chain...
How many unique movies start with the letter B?
SQLQuery:SELECT COUNT(DISTINCT `Movie`) AS UniqueMoviesStartingWithB
FROM telugumovies_dataset
WHERE `Movie` LIKE 'B%'
SQLResult: [(89,)]
Answer:There are 89 unique movies that start with the letter B.
> Finished chain.


C:\Users\lavanya\AppData\Local\Temp\ipykernel_18792\3939423401.py:14: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages).content


'There are 89 unique movies in the database that start with the letter B.'

In [9]:
generate("Tell me the name of the 2 movies which have highest runtime")



> Entering new SQLDatabaseChain chain...
Tell me the name of the 2 movies which have highest runtime
SQLQuery:SELECT `Movie`, `Runtime` 
FROM telugumovies_dataset 
ORDER BY `Runtime` DESC 
LIMIT 2;
SQLResult: [('Athidhi', 300), ('Veera', 240)]
Answer:The names of the 2 movies with the highest runtime are 'Athidhi' and 'Veera'.
> Finished chain.


"The names of the 2 movies with the highest runtime are 'Athidhi' and 'Veera'."

In [11]:
generate("Tell me the name of the 2 movies which have highest ratings.Mention their ratings as well.")



> Entering new SQLDatabaseChain chain...
Tell me the name of the 2 movies which have highest ratings.Mention their ratings as well.
SQLQuery:SELECT `Movie`, `Rating` 
FROM telugumovies_dataset 
ORDER BY `Rating` DESC 
LIMIT 2;
SQLResult: [('Pichhodu', 9.5), ('Maya Bazaar', 9.2)]
Answer:The names of the 2 movies with the highest ratings are 'Pichhodu' with a rating of 9.5 and 'Maya Bazaar' with a rating of 9.2.
> Finished chain.


"The names of the 2 movies with the highest ratings are 'Pichhodu' with a rating of 9.5 and 'Maya Bazaar' with a rating of 9.2."

In [13]:
generate("Tell me the name of the movie and its ratings which has highest ratings and certificate is U.")



> Entering new SQLDatabaseChain chain...
Tell me the name of the movie and its ratings which has highest ratings and certificate is U.
SQLQuery:SELECT `Movie`, `Rating` 
FROM telugumovies_dataset 
WHERE `Certificate` = 'U' 
ORDER BY `Rating` DESC 
LIMIT 1;
SQLResult: [('Maya Bazaar', 9.2)]
Answer:Maya Bazaar with a rating of 9.2 has the highest rating and a certificate of U.
> Finished chain.


'The movie with the highest rating of 9.2 and a U certificate is "Maya Bazaar".'